In [668]:
import numpy as np
import random
import json
from typing import Dict, List, Iterator, Literal, Union, Set, Optional
from __future__ import annotations
from pprint import pprint

np.set_printoptions(
    suppress=True,
    linewidth=180
)

In [669]:
class Logger:
    def __init__(self, name: str, level: Literal["debug","info","warn","error"] = "debug") -> None:
        self.name = name
        self.level = level
    def write(self, level: str, msg: str) -> None:
        print(f"{level.upper()} [{self.name}] " + msg)
    def debug(self, msg: str) -> None:
        if not self.level == "debug": return
        self.write("debug", msg)
    def info(self, msg: str) -> None:
        if self.level not in ["debug","info"]: return
        self.write("info", msg)
    def warn(self, msg: str) -> None:
        if self.level == "error": return
        self.write("warn", msg)
    def error(self, msg: str) -> None:
        self.write("error", msg)

In [670]:
class Vulnerability:
    id: int
    attackerCost: int
    defenderCost: int
    attackerProb: float
    defenderProb: float
    severity: int
    def __init__(
        self,
        id: int,
        acost: Optional[int] = None,
        dcost: Optional[int] = None,
        aprob: Optional[float] = None,
        dprob: Optional[float] = None,
        sev: Optional[int] = None
    ) -> None:
        self.id = id
        if acost is None: acost = random.randint(1,5)
        if dcost is None: dcost = random.randint(1,5)
        if aprob is None: aprob = round(random.uniform(0.05,0.98),2)
        if dprob is None: dprob = round(random.uniform(0.8,0.98),2)
        if sev is None: sev = random.randint(1,5)
        self.attackerCost = acost
        self.defenderCost = dcost
        self.attackerProb = aprob
        self.defenderProb = dprob
        self.severity = sev
    def __str__(self) -> str:
        return f"Action(id={self.id}, acost={self.attackerCost}, dcost={self.defenderCost}, aprob={self.attackerProb}, dprob={self.defenderProb}, sev={self.severity})"
    def __repr__(self) -> str:
        return self.__str__()


In [671]:

class Vehicle:
    id: int
    risk: float
    severity_chances: Dict[int,float]
    in_platoon: bool
    dchoice: List[Vulnerability]
    achoice: List[Vulnerability]
    compromises: Set[Vulnerability]
    known_compromises: Set[Vulnerability]
    def __init__(
        self,
        id,
        risk: float,
        dchoice: List[Vulnerability],
        achoice: List[Vulnerability],
        sevs: Dict[int,float],
        in_platoon: bool = False
    ) -> None:
        self.id = id
        self.risk = risk
        self.dchoice = dchoice
        self.achoice = achoice
        self.severity_chances = sevs
        self.in_platoon = in_platoon
        self.compromises = set()
        self.known_compromises = set()
    def __str__(self) -> str:
        return f"Vehicle(risk={self.risk}, sevs={self.severity_chances}, in_platoon={self.in_platoon})"
    def __repr__(self) -> str:
        return self.__str__()


In [672]:
class VehicleProvider:
    def next(self) -> Vehicle:
        pass
    def reset(self) -> None:
        pass

class JsonVehicleProvider(VehicleProvider):
    vehicles: List[Vehicle]
    seen: Set[Vehicle]
    logger: Logger
    def __init__(self, logger: Logger, path: str) -> None:
        self.logger = logger
        self.seen = set()
        with open(path, "r") as f:
            loaded = json.load(f)
        self.logger.info(f"loaded {len(loaded)} vehicles")
        self.vehicles = []
        id = 0
        for v in loaded:
            self.vehicles.append(Vehicle(
                id,
                -1 * v["defender_util"],
                [Vulnerability(v["id"], v["attackerCost"], v["defenderCost"], v["attackerProb"], v["defenderProb"], v["severity"]) for v in v["dchoice"]],
                [Vulnerability(v["id"], v["attackerCost"], v["defenderCost"], v["attackerProb"], v["defenderProb"], v["severity"]) for v in v["achoice"]],
                {i:v["severity_chances"][str(i)] for i in range(1,6)}
            ))
            id += 1
    def next(self) -> Vehicle:
        candidates = [v for v in self.vehicles if v not in self.seen]
        if len(candidates) == 0: raise Exception("out of vehicles")
        rtn = random.choice(candidates)
        self.seen.add(rtn)
        return rtn

    def reset(self) -> None:
        self.logger.info("resetting json vehicle provider")
        self.i = 0


In [673]:
class Game:
    vehicles: List[Vehicle]
    vehicle_provider: lambda: Vehicle
    attacker_agent: "Agent"
    defender_agent: "Agent"
    logger: Logger
    turn: Literal["defender","attacker"]
    step_count: int
    max_vehicles: int
    def __init__(self, logger: Logger, max_vehicles: int, vehicle_provider: VehicleProvider, defender: "Agent", attacker: "Agent") -> None:
        self.vehicle_provider = vehicle_provider
        self.logger = logger
        self.defender_agent = defender
        self.attacker_agent = attacker
        self.step_count = 0
        self.max_vehicles = max_vehicles
        self.reset()

    def reset(self) -> None:
        self.logger.info("resetting game")
        self.turn = "defender"
        self.vehicles = []
        for _ in range(self.max_vehicles):
            self.vehicles.append(self.vehicle_provider.next())

    def step(self) -> None:
        self.logger.debug("stepping")
        self.turn = "defender" if self.turn == "attacker" else "attacker"
        if self.turn == "attacker":
            self.logger.debug(f"attacker turn begin")
            self.attacker_agent.take_turn(self)
            self.logger.debug(f"attacker turn end")
        if self.turn == "defender":
            self.logger.debug(f"defender turn begin")
            self.defender_agent.take_turn(self)
            self.logger.debug(f"defender turn end")
        
        if self.step_count % 3 == 0 and len(self.vehicles) < self.max_vehicles:
            self.logger.info("cycling out vehicles")
            cycle = 2
            removed = 0
            # candidates = [v for v in self.vehicles if not v.in_platoon]
            candidates = [v for v in self.vehicles]
            random.shuffle(candidates)
            while removed < cycle and len(candidates) > 0:
                v = candidates.pop()
                self.logger.info(f"Vehicle {v.id} risk {v.risk} sev {[x.severity for x in v.compromises]} has left the game.")
                self.vehicles.remove(v)
                removed += 1
            while removed > 0:
                new = self.vehicle_provider.next()
                self.logger.info(f"Vehicle {new.id} risk {new.risk} has joined the game!")
                self.vehicles.append(new)
                removed -= 1
        self.vehicles.index
        self.step_count += 1

In [674]:
class Agent:
    utility: int
    logger: Logger
    def __init__(self, logger: Logger) -> None:
        self.utility = 0
        self.logger = logger
    def take_turn(self, game: Game) -> None:
        pass
    
    def get_utility(self, game: Game) -> int:
        return None

In [675]:
# Very simple agent for defender
class BasicDefenderAgent(Agent):
    num_vehicles_monitoring_constraint: int
    monitoring: Vehicle
    recently_monitored: List[Vehicle]
    tolerance_threshold: int
    max_history: int
    max_size: int
    def __init__(self, logger: Logger) -> None:
        super().__init__(logger)
        self.recently_monitored = []
        self.monitoring = None
        self.utility = 0
        self.tolerance_threshold = 3
        self.max_history = 3
        self.max_size = 10
    
    def get_utility(self, game: Game) -> None:
        members = len([v for v in game.vehicles if v.in_platoon])
        compromises = -sum([s.severity for v in game.vehicles for s in v.compromises ])
        return members * 10 + compromises
    
    def take_turn(self, game: Game) -> None:
        if game.turn != "defender": self.logger.warn("sanity check failed, defender taking turn when game says not their turn")

        # finish monitoring vehicle
        m = self.monitoring
        if m is not None and m.in_platoon:
            # The current compromises become known
            for c in m.compromises:
                if c in m.known_compromises: continue
                self.logger.info(f"discovered compromise on vehicle {m.id} vuln {c.id} sev {c.severity}")
                m.known_compromises.add(c)
            # If beyond threshold, kicked from platoon
            total = sum([s.severity for s in m.known_compromises])
            if total > self.tolerance_threshold:
                self.logger.info(f"kicking vehicle {m.id} out of platoon, severity {total} exceeded threshold {self.tolerance_threshold}")
                m.in_platoon = False

        # pick next vehicle to monitor
        choices = [
            v for v in game.vehicles
            if v not in self.recently_monitored
        ]
        if len(choices) == 0:
            self.logger.warn("no candidates found to monitor?")
            pprint(game.vehicles)
            pprint(self.recently_monitored)
            self.monitoring = None
        else:
            self.monitoring = random.choice(choices)
            self.logger.debug(f"now monitoring vehicle id {self.monitoring.id}")


        # avoid monitoring the last 3 vehicles again
        if len(self.recently_monitored) >= self.max_history:
            self.recently_monitored.pop(0)
        self.recently_monitored.append(m)

        # allow low risk vehicles into the platoon
        candidates = [
            v for v in game.vehicles
            if v.risk <= 10 \
            and sum([s.severity for s in v.known_compromises]) <= self.tolerance_threshold \
            and len([v for v in game.vehicles if v.in_platoon]) < self.max_size
        ]

        # take while there's room in the platoon
        random.shuffle(candidates)
        while len([v for v in game.vehicles if v.in_platoon]) < self.max_size and len(candidates) > 0:
            candidates.pop().in_platoon = True
        
        # calculate util change
        change = self.get_utility(game)
        self.utility += change
        self.logger.debug(f"utility {self.utility} ({change:+d})")


In [676]:
# Very simple agent for attacker
class BasicAttackerAgent(Agent):
    def get_utility(self, game: Game) -> None:
        util = 0
        for v in game.vehicles:
            for c in v.compromises:
                if v.in_platoon:
                    util += c.severity * 2
                else:
                    util += c.severity
        return util

    def take_turn(self, game: Game) -> None:
        if game.turn != "attacker": self.logger.warn("sanity check failed, attacker taking turn when game says not their turn")
        
        # Pick a vehicle to attack.
        # In this case, the vehicles being attacked are always in the platoon.
        # A better agent would attack ones outside as well, there's no rule against it.
        candidates = [v for v in game.vehicles if v.in_platoon]
        def sort(v: Vehicle) -> float:
            return [x.severity ** 2 * x.attackerProb * (1 if not x in v.dchoice else 1 - x.defenderProb) for x in v.achoice if x not in v.compromises]
        candidates = sorted(candidates, key=sort)
        if len(candidates) == 0:
            candidates = game.vehicles
        if len(candidates) == 0:
            self.logger.warn("sanity check failed, no vehicles to attack")
            pprint(game.vehicles)

        # pick victim to attack
        # attack vulnerabilities according to solution of subgame (achoice, dchoice)
        if len(candidates) > 0:
            victim = candidates.pop()
            self.logger.debug(f"attacking vehicle id {victim.id}")
            for vuln in victim.achoice:
                success = random.randint(1,1000) / 1000 > vuln.attackerProb
                if success and vuln in victim.dchoice:
                    success = random.randint(1,1000) / 1000 > vuln.defenderProb
                if success:
                    self.logger.info(f"successfully compromised vehicle {victim.id} vuln {vuln.id} sev {vuln.severity}")
                    victim.compromises.add(vuln)
        
        # calculate util change
        change = self.get_utility(game)
        self.utility += change
        self.logger.debug(f"utility {self.utility} ({change:+d})")


In [677]:
log_level = "debug"
game = Game(
    Logger("Game", log_level),
    10,
    JsonVehicleProvider(Logger("JsonVehicleProvider", log_level), "../subgame/python/solutions.json"),
    BasicDefenderAgent(Logger("BasicDefenderAgent", log_level)),
    BasicAttackerAgent(Logger("BasicAttackerAgent", log_level))
)
dutils = []
autils = []
compromisePctPart = []
compromisePctOverall = []
num_rounds = 100
vcount = []
pcount = []
for i in range(num_rounds):
    dutils.append(game.defender_agent.utility)
    autils.append(game.attacker_agent.utility)
    compromisePctPart.append(len([v for v in game.vehicles if len(v.compromises) > 0])/(len(game.vehicles) if len(game.vehicles) > 0 else 1))
    bot = sum([len(v.achoice) for v in game.vehicles])
    if bot == 0: bot = 1
    compromisePctOverall.append(sum([len(v.compromises) for v in game.vehicles]) / bot)
    vcount.append(len(game.vehicles))
    pcount.append(len([v for v in game.vehicles if v.in_platoon]))
    game.step()

INFO [JsonVehicleProvider] loaded 1000 vehicles
INFO [Game] resetting game
DEBUG [Game] stepping
DEBUG [Game] attacker turn begin
DEBUG [BasicAttackerAgent] attacking vehicle id 939
INFO [BasicAttackerAgent] successfully compromised vehicle 939 vuln 2 sev 2
INFO [BasicAttackerAgent] successfully compromised vehicle 939 vuln 7 sev 5
DEBUG [BasicAttackerAgent] utility 0 (+0)
DEBUG [Game] attacker turn end
INFO [Game] cycling out vehicles
INFO [Game] Vehicle 20 risk 24.95 sev [] has left the game.
INFO [Game] Vehicle 567 risk 8.24 sev [] has left the game.
INFO [Game] Vehicle 640 risk 7.34 has joined the game!
INFO [Game] Vehicle 375 risk 11.02 has joined the game!
DEBUG [Game] stepping
DEBUG [Game] defender turn begin
DEBUG [BasicDefenderAgent] now monitoring vehicle id 245
DEBUG [BasicDefenderAgent] utility 30 (+30)
DEBUG [Game] defender turn end
DEBUG [Game] stepping
DEBUG [Game] attacker turn begin
DEBUG [BasicAttackerAgent] attacking vehicle id 640
INFO [BasicAttackerAgent] successfu

In [678]:
print(game.defender_agent.utility)
pprint(game.vehicles)

700
[Vehicle(risk=5.98, sevs={1: 0, 2: 0, 3: 0, 4: 0.25, 5: 0.07}, in_platoon=True),
 Vehicle(risk=18.2, sevs={1: 0, 2: 0, 3: 0.82, 4: 0.13, 5: 0.2}, in_platoon=False),
 Vehicle(risk=19.44, sevs={1: 0, 2: 0.83, 3: 0, 4: 0.71, 5: 0.19}, in_platoon=False),
 Vehicle(risk=13.57, sevs={1: 0, 2: 0, 3: 0.42, 4: 0, 5: 0.35}, in_platoon=False),
 Vehicle(risk=5.69, sevs={1: 0, 2: 0.57, 3: 0.19, 4: 0, 5: 0.07}, in_platoon=False)]


In [699]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(4, figsize=(16,9), gridspec_kw={'height_ratios': [1, 2, 1, 1]})
i=0
axs[i].plot(dutils, label="defender")
axs[i].plot(autils, label="attacker")
axs[i].legend(loc="upper left")
axs[i].title.set_text("utilities")

i+=1
axs[i].plot(np.diff(dutils), label="defender", alpha=0.8)
axs[i].plot(np.diff(autils), label="attacker", alpha=0.1)
axs[i].legend(loc="upper left")
axs[i].title.set_text("utility deltas")

i+=1
axs[i].plot(compromisePctOverall, label="overall", alpha=0.5)
axs[i].plot(compromisePctPart, label="partly", alpha=1)
axs[i].set_yticks(np.arange(0,1,0.1))
axs[i].title.set_text("fraction compromised")
axs[i].legend(loc="upper left")

i+=1
axs[i].plot(vcount,label="vehicles count")
axs[i].plot(pcount,label="platoon size")
axs[i].legend(loc="upper left")
axs[i].title.set_text("counts")

plt.tight_layout()
plt.subplots_adjust(top=2)